In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from tqdm import tqdm
import os
import time


In [2]:
# 读取数据
df_19 = pd.read_csv('data/19_science/19_science_sorted.csv', )
df_18 = pd.read_excel('data/18_science/18 science_original_chem.xlsx')
df_sm = pd.read_excel("data/suzuki/suzuki.xlsx").dropna().reset_index(drop=True)

# read descriptor
des_path = 'HTE_descriptors'

def read_des(folder='folder_name'):
    # folder = 'MFP_descriptor'
    file_list = [f for f in os.listdir(folder) if f.endswith('.csv')]
    # 读取并处理每个文件
    fp_dfs = []
    for filename in file_list:
        filepath = os.path.join(folder, filename)
        df = pd.read_csv(filepath)

        if 'Original_SMILES' in df.columns:
            df = df.drop(columns=['Original_SMILES'])
    
        # 使用文件名作为前缀（去掉扩展名）
        prefix = os.path.splitext(filename)[0]
        df.columns = [f"{prefix}_{col}" for col in df.columns]
        print(f"{filename}的维度:", df.shape)
        fp_dfs.append(df)

    # 合并所有分子指纹特征
    combined_fp = pd.concat(fp_dfs, axis=1)
    # 输出结果查看
    print("合并后的维度:", combined_fp.shape)
    return combined_fp

# one-hot encode
OHE_19 = pd.read_csv(os.path.join(des_path, f"OH_encode/19_onehot.csv"))
OHE_18 = pd.read_csv(os.path.join(des_path, f"OH_encode/18_onehot.csv"))
OHE_sm = pd.read_csv(os.path.join(des_path, f"OH_encode/sm_onehot.csv"))

print(f"OHE19:{OHE_19.shape}, OHE18:{OHE_18.shape}, OHE_sm:{OHE_sm.shape}")

# 分子指纹（Morgan Fingerprint）

# FP_19 = read_des(os.path.join(des_path, 'MFP_descriptor/19_data'))
# FP_18 = read_des(os.path.join(des_path, 'MFP_descriptor/18_data'))
# FP_sm = read_des(os.path.join(des_path, 'MFP_descriptor/suzuki'))
FP_19 = read_des(os.path.join(des_path, 'MACCS/19_data'))
FP_18 = read_des(os.path.join(des_path, 'MACCS/18_data'))
FP_sm = read_des(os.path.join(des_path, 'MACCS/suzuki'))

# RDKit 分子性质
RDkit_19 = read_des(os.path.join(des_path, 'RDKit_descriptors/19_data'))
RDKit_18 = read_des(os.path.join(des_path, 'RDKit_descriptors/18_data'))
RDkit_sm = read_des(os.path.join(des_path, 'RDKit_descriptors/suzuki'))

# mordred
Mord_19 = read_des(os.path.join(des_path, 'Mordred/19_data'))
Mord_18 = read_des(os.path.join(des_path, 'Mordred/18_data'))
Mord_sm = read_des(os.path.join(des_path, 'Mordred/suzuki'))

# unimol
UniMol_19 = read_des(os.path.join(des_path, 'unimol/19_data'))
UniMol_18 = read_des(os.path.join(des_path, 'unimol/18_data'))
UniMol_sm = read_des(os.path.join(des_path, 'unimol/suzuki'))

# X_desc = rdkit_descriptors(df['full_smi'])
# desc_dim = X_desc.shape[1]

OHE19:(1075, 53), OHE18:(3955, 44), OHE_sm:(4620, 33)
Catalyst_smi_MACCS.csv的维度: (1075, 167)
Imine_smi_MACCS.csv的维度: (1075, 167)
MACCS.csv的维度: (5, 167)
Thiol_smi_MACCS.csv的维度: (1075, 167)
合并后的维度: (1075, 668)
Additive_MACCS.csv的维度: (3955, 167)
Aryl halide_MACCS.csv的维度: (3955, 167)
Base_MACCS.csv的维度: (3955, 167)
Ligand_MACCS.csv的维度: (3955, 167)
合并后的维度: (3955, 668)
ligand_fingerprints.csv的维度: (4620, 167)
reactant_1_fingerprints.csv的维度: (4620, 167)
reactant_2_fingerprints.csv的维度: (4620, 167)
reagent_1_fingerprints.csv的维度: (4620, 167)
solvent_1_fingerprints.csv的维度: (4620, 167)
合并后的维度: (4620, 835)
Catalyst_smi_descriptors.csv的维度: (1075, 209)
Imine_smi_descriptors.csv的维度: (1075, 209)
Thiol_smi_descriptors.csv的维度: (1075, 209)
合并后的维度: (1075, 627)
Additive_descriptors.csv的维度: (3955, 209)
Aryl halide_descriptors.csv的维度: (3955, 209)
Base_descriptors.csv的维度: (3955, 209)
Ligand_descriptors.csv的维度: (3955, 209)
合并后的维度: (3955, 836)
ligand_descriptors.csv的维度: (4620, 209)
reactant_1_descriptors.csv的维度: (

In [31]:
# 清理 RDKit 特征数据
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

def clean_rdkit_features(X):
    imp = SimpleImputer(strategy='mean')
    scaler = StandardScaler()
    X_clean = imp.fit_transform(X)
    X_scaled = scaler.fit_transform(X_clean)
    return X_scaled

def evaluate_rf(X, y, n_runs=30, n_estimators=500, max_depth=25, min_samples_split=2):
    r2_scores, maes, times = [], [], []

    for _ in tqdm(range(n_runs)):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            random_state=42,
            n_jobs=-1
        )

        start = time.time()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        duration = time.time() - start

        r2_scores.append(r2_score(y_test, y_pred))
        maes.append(mean_absolute_error(y_test, y_pred))
        times.append(duration)

    return {
        'R2_mean': np.mean(r2_scores),
        'R2_std': np.std(r2_scores),
        'MAE_mean': np.mean(maes),
        'MAE_std': np.std(maes),
        'Time_mean': np.mean(times),
        'Time_std': np.std(times)
    }

In [54]:
results = []
#运行对比 只使用随机森林模型
# load data
X_onehot = OHE_18
X_fp = FP_18
X_desc = RDKit_18
X_mord = Mord_18
X_unimol = UniMol_18
y = df_18['Output'].values

features = {
    # "OneHot_SMILES": (X_onehot, {"n_estimators": 300, "max_depth": 10, "min_samples_split": 2}),
    # "Fingerprint": (X_fp, {"n_estimators": 300, "max_depth": 20, "min_samples_split": 4}),
    # "RDKit_Descriptors": (X_desc, {"n_estimators": 300, "max_depth": 15, "min_samples_split": 3}),
    # "Mordred": (X_mord, {"n_estimators": 300, "max_depth": 18, "min_samples_split": 3}),
    "Uni-Mol": (X_unimol, {"n_estimators": 300, "max_depth": 15, "min_samples_split": 2}),
}

results = []

for name, (Xf, params) in features.items():
    result = evaluate_rf(Xf, y, **params)
    results.append([
        name, Xf.shape[1], result['R2_mean'], result['R2_std'],
        result['MAE_mean'], result['MAE_std'], result['Time_mean'], result['Time_std']
    ])

result_df = pd.DataFrame(results, columns=[
    "Feature_Type", "Dim",
    "R2_mean", "R2_std", "MAE_mean", "MAE_std","Time_mean", "Time_std"
])
print(result_df)
# result_df.to_csv('results/ADD/18_science_RF.csv')

  0%|          | 0/30 [00:00<?, ?it/s]

 10%|█         | 3/30 [05:24<48:40, 108.17s/it]


KeyboardInterrupt: 

In [37]:
results = []
#运行对比 只使用随机森林模型
# load data
X_onehot = OHE_sm
X_fp = FP_sm
X_desc = RDkit_sm
X_mord = Mord_sm
X_unimol = UniMol_sm
y = df_sm['Output'].values

features = {
    "OneHot_SMILES": (X_onehot, {"n_estimators": 300, "max_depth": 10, "min_samples_split": 2}),
    "Fingerprint": (X_fp, {"n_estimators": 300, "max_depth": 20, "min_samples_split": 4}),
    # "RDKit_Descriptors": (X_desc, {"n_estimators": 300, "max_depth": 15, "min_samples_split": 3}),
    "Mordred": (X_mord, {"n_estimators": 300, "max_depth": 18, "min_samples_split": 3}),
    "Uni-Mol": (X_unimol, {"n_estimators": 500, "max_depth": 25, "min_samples_split": 2}),
}

results = []

for name, (Xf, params) in features.items():
    result = evaluate_rf(Xf, y, **params)
    results.append([
        name, Xf.shape[1], result['R2_mean'], result['R2_std'],
        result['MAE_mean'], result['MAE_std'], result['Time_mean'], result['Time_std']
    ])

result_df = pd.DataFrame(results, columns=[
    "Feature_Type", "Dim",
    "R2_mean", "R2_std", "MAE_mean", "MAE_std","Time_mean", "Time_std"
])
print(result_df)
result_df.to_csv('results/ADD/18_science_RF.csv')

100%|██████████| 30/30 [52:59<00:00, 105.98s/it]

    Feature_Type    Dim   R2_mean    R2_std  MAE_mean   MAE_std   Time_mean  \
0  OneHot_SMILES     33  0.767226  0.013337  0.097088  0.002508    0.403916   
1    Fingerprint  10240  0.843946  0.011125  0.076792  0.002454    4.471080   
2        Mordred   5116  0.846380  0.008972  0.076557  0.001874   56.664036   
3        Uni-Mol   2560  0.846607  0.009998  0.076785  0.002237  105.933555   

   Time_std  
0  0.004914  
1  0.170679  
2  0.578813  
3  1.167414  


In [10]:
# Define path and csv files
DATA_DIR = './data/19_science/'
# OUT_DIR = 'out/models_unimol_infer'+datetime.now().strftime('%y%m%d%H%M')+'/'
# 分子特征
INPUTS_Catlyst_repr = 'Catalyst_smirepr.csv'  # Unscaled  data 
INPUTS_Imine_repr = 'Imine_smirepr.csv'
INPUTS_Thiol_repr = 'Thiol_smirepr.csv'
INPUTS_Origin_DF = '19_science_total.csv'

inputs_Catlyst_repr = pd.read_csv(DATA_DIR + INPUTS_Catlyst_repr)
inputs_Imine_repr = pd.read_csv(DATA_DIR + INPUTS_Imine_repr)
inputs_Thiol_repr = pd.read_csv(DATA_DIR + INPUTS_Thiol_repr)
delta_G = pd.read_csv(DATA_DIR + INPUTS_Origin_DF)['Output']

inputs = np.concatenate([inputs_Catlyst_repr,inputs_Imine_repr,inputs_Thiol_repr],axis=1)

In [11]:
import random
import torch


new_split = 1

predictions = []
r2_values = []
rmse_values = []

OUT_DIR = 'out/19_science/'
for _ in tqdm(range(10)):
    if new_split == 1:
        # 随机生成train和test的indices
        total_cleaned_len = len(inputs_Catlyst_repr)
        train_index = random.sample(range(total_cleaned_len), int(600))#论文中用了600个作为训练集 int(1075*0.7)
        train_indices = np.zeros((total_cleaned_len)).astype(np.bool_)
        for tcl in range(total_cleaned_len):
            if tcl in train_index:
                train_indices[tcl] = True
        test_indices = ~ train_indices
        np.savetxt(OUT_DIR + 'clean_data_train_indices.csv', train_indices, delimiter = ',')
        np.savetxt(OUT_DIR + 'clean_data_test_indices.csv', test_indices, delimiter = ',')

    elif new_split == 0:
        # read saved train indices and test indices
        train_indices = np.loadtxt(OUT_DIR + 'clean_data_train_indices.csv',dtype='bool', delimiter=',')
        test_indices = np.loadtxt(OUT_DIR + 'clean_data_test_indices.csv',dtype='bool', delimiter=',')

    # Load yield data
    delta2G = np.array(delta_G)
    delta2G = delta2G.flatten()
    delta2G = np.nan_to_num(delta2G, nan=0)
    # print('len(inputs): ', len(inputs))
    # Use the indices to generate train/test sets
    X_train = inputs[train_indices]
    #X_train = inputs[:2]
    y_train = delta2G[train_indices]
    featuresTrain = torch.from_numpy(X_train)
    targetsTrain = torch.from_numpy(y_train)
    batch_size = len(X_train)
    # print('batch_size: ', batch_size)

    X_test = inputs[test_indices]
    y_test = delta2G[test_indices]
    featuresTest = torch.from_numpy(X_test)
    targetsTest = torch.from_numpy(y_test)#.type(torch.LongTensor)
    batch_size_test = len(X_test)
    # print('batch_size_test: ', batch_size_test)

    train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
    test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

    train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
    test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size_test, shuffle = False)

    model =  RandomForestRegressor(n_estimators=300,random_state=42, max_depth=25, min_samples_split=2)
    model.fit(X_train, y_train.ravel())
    preds = model.predict(X_test)

    # calculate an R-squared and RMSE values
    r_squared = r2_score(y_test, preds)
    r2_values.append(r_squared)
    mae = mean_absolute_error(y_test, preds)
    rmse_values.append(mae)
# print(r_squared, mae)
print(f'R2_mean:{np.mean(r2_values)}' ,f'R2_std: {np.std(r2_values)}',f'MAE_mean: {np.mean(rmse_values)}',f'MAE_std: {np.std(rmse_values)}')

100%|██████████| 10/10 [19:59<00:00, 120.00s/it]

R2_mean:0.9065780622226542 R2_std: 0.007746701650625964 MAE_mean: 0.14800891649328551 MAE_std: 0.006035723749478535


# 18_science Buchwald

In [16]:
new_split = 1

predictions = []
r2_values = []
rmse_values = []

DATA_DIR = './data/18_science/'
# OUT_DIR = 'out/models_unimol_infer'+datetime.now().strftime('%y%m%d%H%M')+'/'

INPUTS_Aryl_halide_repr = 'Aryl haliderepr.csv'  # Unscaled  data 
INPUTS_Additive_repr = 'Additiverepr.csv'
INPUTS_Base_repr = 'Baserepr.csv'
INPUTS_Ligand_repr = 'Ligandrepr.csv'
INPUTS_Origin_DF = '18 science_original_chem.xlsx'

inputs_Aryl_halide_repr = pd.read_csv(DATA_DIR + INPUTS_Aryl_halide_repr)
inputs_Additive_repr = pd.read_csv(DATA_DIR + INPUTS_Additive_repr)
inputs_Base_repr = pd.read_csv(DATA_DIR + INPUTS_Base_repr)
inputs_Ligand_repr = pd.read_csv(DATA_DIR + INPUTS_Ligand_repr)
yields = pd.read_excel(DATA_DIR + INPUTS_Origin_DF)['Output']

inputs = np.concatenate([inputs_Aryl_halide_repr ,inputs_Additive_repr,inputs_Base_repr,inputs_Ligand_repr],axis=1)

OUT_DIR = 'out/18_science/'
for _ in tqdm(range(30)):
    if new_split == 1:
        # 随机生成train和test的indices
        total_cleaned_len = len(inputs_Aryl_halide_repr)
        train_index = random.sample(range(total_cleaned_len), int(3955*0.7))
        train_indices = np.zeros((total_cleaned_len)).astype(np.bool_)
        for tcl in range(total_cleaned_len):
            if tcl in train_index:
                train_indices[tcl] = True
        test_indices = ~ train_indices
        np.savetxt(OUT_DIR + 'clean_data_train_indices.csv', train_indices, delimiter = ',')
        np.savetxt(OUT_DIR + 'clean_data_test_indices.csv', test_indices, delimiter = ',')

    elif new_split == 0:
        # read saved train indices and test indices
        train_indices = np.loadtxt(OUT_DIR + 'clean_data_train_indices.csv',dtype='bool', delimiter=',')
        test_indices = np.loadtxt(OUT_DIR + 'clean_data_test_indices.csv',dtype='bool', delimiter=',')

    # Load yield data
    yields = np.array(yields)
    yields = yields.flatten()
    yields = np.nan_to_num(yields, nan=0)
    # print('len(inputs): ', len(inputs))
    # Use the indices to generate train/test sets
    X_train = inputs[train_indices]
    #X_train = inputs[:2]
    y_train = yields[train_indices]
    featuresTrain = torch.from_numpy(X_train)
    targetsTrain = torch.from_numpy(y_train)
    batch_size = len(X_train)
    # print('batch_size: ', batch_size)

    X_test = inputs[test_indices]
    y_test = yields[test_indices]
    featuresTest = torch.from_numpy(X_test)
    targetsTest = torch.from_numpy(y_test)#.type(torch.LongTensor)
    batch_size_test = len(X_test)
    # print('batch_size_test: ', batch_size_test)

    train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
    test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

    train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
    test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size_test, shuffle = False)

    model =  RandomForestRegressor(n_estimators=300,random_state=42, max_depth=25, min_samples_split=2)
    model.fit(X_train, y_train.ravel())
    preds = model.predict(X_test)

    # calculate an R-squared and RMSE values
    r_squared = r2_score(y_test, preds)
    r2_values.append(r_squared)
    mae = mean_absolute_error(y_test, preds)
    rmse_values.append(mae)
# print(r_squared, mae)
print(f'R2_mean:{np.mean(r2_values)}' ,f'R2_std: {np.std(r2_values)}',f'MAE_mean: {np.mean(rmse_values)}',f'MAE_std: {np.std(rmse_values)}')

100%|██████████| 30/30 [4:06:43<00:00, 493.45s/it]  

R2_mean:0.9198322302837165 R2_std: 0.009513109441769594 MAE_mean: 5.06341686222863 MAE_std: 0.19155827265144476


## suzuki

In [17]:
new_split = 1

predictions = []
r2_values = []
rmse_values = []

DATA_DIR = './data/suzuki/'

if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
# 分子特征
INPUTS_reactant1_repr = 'reactant_1repr.csv'  # Unscaled  data 
INPUTS_reactant2_repr = 'reactant_2repr.csv'
INPUTS_reagent1_repr = 'reagent_1repr.csv'
INPUTS_solvent1_repr = 'solvent_1repr.csv'
INPUTS_ligand_repr = 'ligandrepr.csv'
INPUTS_Origin_DF = 'suzuki.xlsx'

inputs_reactant1_repr = pd.read_csv(DATA_DIR + INPUTS_reactant1_repr)
inputs_reactant2_repr = pd.read_csv(DATA_DIR + INPUTS_reactant2_repr)
inputs_reagent1_repr = pd.read_csv(DATA_DIR + INPUTS_reagent1_repr)
inputs_solvent1_repr = pd.read_csv(DATA_DIR + INPUTS_solvent1_repr)
inputs_Ligand_repr = pd.read_csv(DATA_DIR + INPUTS_ligand_repr)
df_ori = pd.read_excel(DATA_DIR + INPUTS_Origin_DF)
df = df_ori.dropna(axis=0, how='any')
inputs = np.concatenate([inputs_reactant1_repr ,inputs_reactant2_repr,inputs_reagent1_repr,inputs_solvent1_repr,inputs_Ligand_repr],axis=1)
yields = df['Output']


OUT_DIR = 'out/suzuki/'
for _ in tqdm(range(30)):
    if new_split == 1:
        # 随机生成train和test的indices
        total_cleaned_len = len(inputs_reactant1_repr)
        train_index = random.sample(range(total_cleaned_len), int(total_cleaned_len*0.7))
        train_indices = np.zeros((total_cleaned_len)).astype(np.bool_)
        for tcl in range(total_cleaned_len):
            if tcl in train_index:
                train_indices[tcl] = True
        test_indices = ~ train_indices
        np.savetxt(OUT_DIR + 'clean_data_train_indices.csv', train_indices, delimiter = ',')
        np.savetxt(OUT_DIR + 'clean_data_test_indices.csv', test_indices, delimiter = ',')

    elif new_split == 0:
        # read saved train indices and test indices
        train_indices = np.loadtxt(OUT_DIR + 'clean_data_train_indices.csv',dtype='bool', delimiter=',')
        test_indices = np.loadtxt(OUT_DIR + 'clean_data_test_indices.csv',dtype='bool', delimiter=',')

    # Load yield data
    yields = np.array(yields)
    yields = yields.flatten()
    yields = np.nan_to_num(yields, nan=0)
    # print('len(inputs): ', len(inputs))
    # Use the indices to generate train/test sets
    X_train = inputs[train_indices]
    #X_train = inputs[:2]
    y_train = yields[train_indices]
    featuresTrain = torch.from_numpy(X_train)
    targetsTrain = torch.from_numpy(y_train)
    batch_size = len(X_train)
    # print('batch_size: ', batch_size)

    X_test = inputs[test_indices]
    y_test = yields[test_indices]
    featuresTest = torch.from_numpy(X_test)
    targetsTest = torch.from_numpy(y_test)#.type(torch.LongTensor)
    batch_size_test = len(X_test)
    # print('batch_size_test: ', batch_size_test)

    train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
    test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

    train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
    test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size_test, shuffle = False)

    model =  RandomForestRegressor(n_estimators=300,random_state=42, max_depth=25, min_samples_split=2)
    model.fit(X_train, y_train.ravel())
    preds = model.predict(X_test)

    # calculate an R-squared and RMSE values
    r_squared = r2_score(y_test, preds)
    r2_values.append(r_squared)
    mae = mean_absolute_error(y_test, preds)
    rmse_values.append(mae)
# print(r_squared, mae)
print(f'R2_mean:{np.mean(r2_values)}' ,f'R2_std: {np.std(r2_values)}',f'MAE_mean: {np.mean(rmse_values)}',f'MAE_std: {np.std(rmse_values)}')

100%|██████████| 30/30 [4:44:39<00:00, 569.31s/it]  

R2_mean:0.8476579457154416 R2_std: 0.008287351257785949 MAE_mean: 0.07645545984141394 MAE_std: 0.002079452394050178
